# Evaluating a Chat Bot (Single-Turn)
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langsmith-cookbook/blob/master/./testing-examples/chat-single-turn/chat_evaluation_single_turn.ipynb)

Chat bots, designed to assist users or provide entertainment, typically engage in multiple conversational turns in any given chat session. Over extended interactions, there is potential for the quality of the LLM responses to diminish. This can manifest as difficulty in recalling earlier conversation details, improper interaction with external resources (if available), or repetitive and uninspired responses. If your current offline evaluations focus solely on individual question-answer pairs, such nuances may go unnoticed.

This walkthrough guides you in creating a dataset to evaluate chatbots within multi-turn conversations. Rather than complicate the evaluation with simulation-based methods, the technique outlined below simplifies the process: we treat each data point as an individual dialogue turn. When these are sourced from different stages of actual dialogues, they effectively capture the typical behaviors your model should exhibit.

This notebook provides a concise overview of the process. Let's get started!

## Prerequisites

This walkthrough uses LangChain and OpenAI. Install these packages below and configure your API keys accordingly.

In [1]:
%pip install -U langchain openai

In [ ]:
import os

os.environ["LANGCHAIN_API_KEY"] = "YOUR API KEY"
os.environ["OPENAI_API_KEY"] = "YOUR API KEY"

In [2]:
import uuid

from langsmith import Client

uid = uuid.uuid4()
client = Client()

# 1. Create dataset

The key component of a multi-turn dialogue dataset is a list of conversational "turns". For each dataset example row,  we will evaluate how well the chat bot can respond, taking the conversation history into account. 

To accomplish his, we will make a dataset using a list of chat message dictionaries. We have factored out the "incoming" user query as a separate field.

In [3]:
examples = [
    {
        "question": "For",
        "chat_history": [
            {
                "role": "user",
                "content": "Help me prepare for my debate",
            },
            {
                "role": "assistant",
                "content": "Sure thing, what is the topic of debate?",
            },
            {
                "role": "user",
                "content": "Use of technology in the classroom.",
            },
            {
                "role": "assistant",
                "content": "Great. Which side would you like to represent?",
            },
        ],
    },
    {
        "question": "Yes, that would be helpful.",
        "chat_history": [
            {
                "role": "user",
                "content": "I’m having trouble understanding cultural relativism.",
            },
            {
                "role": "assistant",
                "content": "It’s a crucial concept. Essentially, it’s about viewing other cultures without the lens of our own biases.",
            },
            {
                "role": "user",
                "content": "So, like not judging other cultures based on our own standards?",
            },
            {
                "role": "assistant",
                "content": "Exactly! It helps anthropologists to understand cultures on their own terms. Would you like a historical example?",
            },
        ],
    },
    {
        "question": "Was the American Revolution an influence too?",
        "chat_history": [
            {
                "role": "user",
                "content": "Can you explain the causes of the French Revolution?",
            },
            {
                "role": "assistant",
                "content": "Certainly. The causes are manifold including economic hardship, social inequality, and political incompetence.",
            },
            {
                "role": "user",
                "content": "The bourgeoisie were a part of the Third Estate, right?",
            },
            {
                "role": "assistant",
                "content": "Correct. They were part of the larger Third Estate which also included peasants and workers, and they were heavily taxed unlike the other estates.",
            },
            {"role": "user", "content": "How did Enlightenment ideas contribute?"},
            {
                "role": "assistant",
                "content": "Enlightenment ideas inspired the populace by promoting rights and equality, which directly challenged the established feudal system.",
            },
        ],
    },
    {
        "question": "And this applies to all actions, no matter the size?",
        "chat_history": [
            {
                "role": "user",
                "content": "I can’t wrap my head around Newton’s Third Law.",
            },
            {
                "role": "assistant",
                "content": "It states that for every action, there’s an equal and opposite reaction. Think of it as a force pair.",
            },
            {"role": "user", "content": "Like when a rocket launches?"},
            {
                "role": "assistant",
                "content": "Exactly! The rocket propels fuel downwards, and in turn, it is propelled upwards with equal force.",
            },
        ],
    },
    {
        "question": "When do I steer left?",
        "chat_history": [
            {"role": "user", "content": "Parallel parking is tricky."},
            {"role": "assistant", "content": "It can be. Would you like some tips?"},
            {"role": "user", "content": "Yes, please."},
            {
                "role": "assistant",
                "content": "Start by aligning your car beside the other car, leaving about 2-3 feet between them.",
            },
            {
                "role": "user",
                "content": "Then steer all the way to the right while backing up?",
            },
            {
                "role": "assistant",
                "content": "Yes, but when your front door passes the other car’s bumper, straighten the wheel and continue to reverse.",
            },
        ],
    },
]

In [4]:
dataset_name = f"Chat Bot Evals Single-Turn Example - {uid}"
dataset = client.create_dataset(dataset_name)
client.create_examples(
    inputs=examples,
    dataset_id=dataset.id,
)

## 2. Define Custom Evaluator

Let's say we want to use the reference free "helpfulness" evaluator. For simple situations like the dataset provided,
we'd expect the helpfulness scores to be very high (close to 1). The evaluator below illustrates how you could take data from multiple input keys (in our case, "question", and "chat_history") and format them for some shared function, in our case, the [score_string](https://python.langchain.com/docs/guides/evaluation/string/scoring_eval_chain) evaluator.

In [5]:
from langchain.evaluation import load_evaluator
from langsmith.evaluation import EvaluationResult, RunEvaluator
from langsmith.schemas import Example, Run


class ConversationalHelpfulnessEvaluator(RunEvaluator):
    def __init__(self):
        self.evaluator = load_evaluator(
            "score_string", criteria="helpfulness", normalize_by=10
        )

    def _format_conversation(self, question: str, chat_history: list):
        return (
            "\n".join([f"{turn['role']}: {turn['content']}" for turn in chat_history])
            + f"User: {question}"
        )

    def evaluate_run(self, run: Run, example: Example):
        # The specifics of the _format_conversation function, including
        # which keys to select, depend on the keys in your dataset.
        formatted_history = self._format_conversation(
            example.inputs["question"], example.inputs["chat_history"]
        )
        prediction = run.outputs["output"]
        eval_result = self.evaluator.evaluate_strings(
            input=formatted_history,
            prediction=prediction,
        )
        return EvaluationResult(key="conversation_helpfulness", **eval_result)

Updateing forward refs


## Step 3: Define Chat Bot

Our simple chat bot just formats the messages for an LLM and responds with the resulting content.

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser

# An example chain
chain = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful tutor AI."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
        ]
    )
    | ChatOpenAI(model="gpt-3.5-turbo")
    | StrOutputParser()
)

## Step 4: Evaluate

Now that we have defined our evaluator and chat bot, it's time to run the evaluation.
We will pass a constructor in to the `run_on_dataset` function to create a new instance
of our bot for each dataset example. This also includes some mapping logic to prepare
the data for the bot itself.

In [7]:
from langchain.adapters.openai import convert_openai_messages
from langchain.smith import RunEvalConfig


def construct_chain():
    # Add a step to convert the data from the dataset to a form the chain can consume
    return {
        "input": lambda x: x["question"],
        "chat_history": lambda x: convert_openai_messages(x["chat_history"]),
    } | chain


results = client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=construct_chain,
    evaluation=RunEvalConfig(
        custom_evaluators=[ConversationalHelpfulnessEvaluator()],
    ),
)

View the evaluation results for project 'test-slight-error-59' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/cd82cfa6-9ee6-4851-894a-788374cc1656?eval=true
[------------------------------------------------->] 5/5

## Conclusion

We've covered the steps to create a conversational dataset, set up a custom evaluator,  and apply that evaluator to your chat bot to measure its ability to handle conversation history.

Effective evaluation focuses on ensuring the bot maintains consistent and accurate interactions across turns. With the methods outlined here, you're set up to measure and improve this aspect of your chat bot's performance.